# Quantum Teleportation: Transferring a Qubit Without Moving It

## 📌 Introduction

Quantum teleportation is a protocol that allows the transfer of an unknown quantum state from one location to another, using entanglement and classical communication. Importantly, the state is not cloned but destroyed at the source and reconstructed at the destination, maintaining the no-cloning theorem of quantum mechanics.

This is one of the most fundamental quantum algorithms because it combines multiple quantum concepts: entanglement, measurement, and the need for classical communication in a quantum world. It lays the groundwork for quantum networks, distributed quantum computing, and quantum cryptography.

## 🎯 Learning Goals

By the end of this tutorial, you will be able to:
- Understand the problem quantum teleportation solves
- Grasp how entanglement and measurement enable teleportation
- Identify how each quantum gate in the circuit contributes to the protocol
- Implement and test the teleportation algorithm using Qiskit
- Run it both on a simulator and on a real quantum backend (optional)

## 📚 Background

To understand quantum teleportation, you need to be familiar with:
- **Qubits**: The fundamental units of quantum information
- **Superposition**: A qubit can exist in a combination of |0⟩ and |1⟩
- **Entanglement**: A strong correlation between two qubits such that the state of one determines the state of the other
- **Quantum Measurement**: Collapses a qubit to |0⟩ or |1⟩ probabilistically
- **Bell State**: An entangled pair (e.g., (|00⟩ + |11⟩)/√2)
- **Quantum Gates**: Especially H (Hadamard), CNOT, X, and Z

## 🔧 Setup: Installing Qiskit

First, let's install Qiskit and its dependencies:

In [ ]:
# Install Qiskit with visualization support
!pip install qiskit[visualization]

In [ ]:
# Verify Qiskit installation
import qiskit
print("Qiskit version:", qiskit.__version__)

## 🧠 Step-by-Step Algorithm Explanation

### 💡 The Idea:
1. You want to teleport the unknown state |ψ⟩ = α|0⟩ + β|1⟩ from Alice to Bob
2. Alice and Bob share an entangled pair of qubits
3. Alice performs quantum operations and measurements on her qubits
4. She sends her measurement results to Bob (classical bits)
5. Bob uses these to reconstruct the original state on his qubit

### 🔧 Circuit Breakdown:
1. Prepare a 3-qubit circuit
2. Qubit 0: |ψ⟩ — the state to teleport
3. Qubits 1 and 2: initialized to |0⟩
4. Entangle qubits 1 and 2
5. Apply CNOT and H to qubit 0
6. Measure qubits 0 and 1
7. Depending on those results, apply X and/or Z to qubit 2

## 🔬 Implementation: Let's Build the Quantum Teleportation Circuit

We'll start by importing the necessary libraries:

In [ ]:
# Import required libraries
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt
import numpy as np

### Step 1: Prepare the Quantum Circuit

First, we'll create the quantum and classical registers, and then build our circuit:

In [ ]:
# Define the quantum and classical registers
# Three qubits: [0] for psi (to teleport), [1] for A (Alice), and [2] for B (Bob)
qreg = QuantumRegister(3, name="q")
# Classical bits to store measurement results
creg = ClassicalRegister(2, name="c")

# Create the quantum circuit
teleport_circuit = QuantumCircuit(qreg, creg)

### Step 2: Prepare the State to Teleport

We'll initialize qubit 0 to a superposition state |+⟩ = (|0⟩ + |1⟩)/√2 using a Hadamard gate, which we'll teleport to Bob:

In [ ]:
# Step 1: Initialize the state to teleport (|+> state in this case)
teleport_circuit.h(qreg[0])
teleport_circuit.barrier()

# Display the circuit so far
teleport_circuit.draw(output='mpl')

### Step 3: Create an Entangled Bell Pair

Next, we need to create an entangled pair between Alice (qubit 1) and Bob (qubit 2):

In [ ]:
# Step 2: Create an entangled Bell pair between qubits A and B
teleport_circuit.h(qreg[1])  # Apply Hadamard to qubit 1 (Alice)
teleport_circuit.cx(qreg[1], qreg[2])  # Apply CNOT with qubit 1 as control and qubit 2 as target
teleport_circuit.barrier()

# Display the circuit so far
teleport_circuit.draw(output='mpl')

### Step 4: Perform Bell Measurement

Now, Alice performs a Bell measurement on her qubits (0 and 1):

In [ ]:
# Step 3: Bell measurement of the psi qubit and qubit A
teleport_circuit.cx(qreg[0], qreg[1])  # CNOT with qubit 0 as control and 1 as target
teleport_circuit.h(qreg[0])  # Apply Hadamard to qubit 0
teleport_circuit.barrier()

# Step 4: Measure qubits 0 and 1
teleport_circuit.measure(qreg[0], creg[0])  # Measure qubit 0 to classical bit 0
teleport_circuit.measure(qreg[1], creg[1])  # Measure qubit 1 to classical bit 1
teleport_circuit.barrier()

# Display the circuit so far
teleport_circuit.draw(output='mpl')

### Step 5: Apply Corrections Based on Measurement Results

Finally, Bob applies quantum gates to reconstruct the state based on Alice's measurement results:

In [ ]:
# Step 5: Apply classical corrections to qubit B based on measurement results
teleport_circuit.x(qreg[2]).c_if(creg[1], 1)  # Apply X gate if qubit A measured as 1
teleport_circuit.z(qreg[2]).c_if(creg[0], 1)  # Apply Z gate if psi qubit measured as 1

# Display the complete circuit
teleport_circuit.draw(output='mpl', fold=20)

## 🔬 Running the Quantum Teleportation Circuit

Now, let's run the teleportation circuit on a simulator:

In [ ]:
# Initialize the simulator
simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit
job = execute(teleport_circuit, simulator, shots=1024)
result = job.result()
counts = result.get_counts(teleport_circuit)

# Display the results
print("Simulation Results:")
print(counts)
plot_histogram(counts)

## 🔍 Verifying the Teleportation

Let's verify that the quantum teleportation worked correctly by checking if Bob's qubit (qubit 2) received the state |+⟩ that we initially prepared on qubit 0.

We'll create a modified circuit that includes a measurement of Bob's qubit after the teleportation:

In [ ]:
# Create a verification circuit
verify_circuit = teleport_circuit.copy()
verify_circuit.barrier()

# Apply Hadamard to transform |+> back to |0> before measurement
verify_circuit.h(qreg[2])

# Add an additional classical register for Bob's measurement
verify_measure = ClassicalRegister(1, 'verify')
verify_circuit.add_register(verify_measure)

# Measure Bob's qubit
verify_circuit.measure(qreg[2], verify_measure[0])

# Draw the verification circuit
verify_circuit.draw(output='mpl', fold=20)

In [ ]:
# Execute the verification circuit
verify_job = execute(verify_circuit, simulator, shots=1024)
verify_result = verify_job.result()
verify_counts = verify_result.get_counts()

# Display the verification results
print("Verification Results:")
print(verify_counts)
plot_histogram(verify_counts)

## 💻 The Math & Algorithms Behind Quantum Teleportation

### 1. Core Quantum Mechanical Principles

#### Quantum Superposition
- **Mathematical representation**: A qubit |ψ⟩ can exist in a state |ψ⟩ = α|0⟩ + β|1⟩ where α and β are complex amplitudes with |α|² + |β|² = 1
- **In the code**: Created with the Hadamard gate (H) applied to |0⟩, producing the state |+⟩ = (|0⟩ + |1⟩)/√2
- **Line of code**: `teleport_circuit.h(qreg[0])` initializes the state to teleport

#### Quantum Entanglement
- **Mathematical representation**: Creation of the Bell state |Φ⁺⟩ = (|00⟩ + |11⟩)/√2
- **In the code**: Created with H gate followed by CNOT gate
- **Lines of code**: 
  ```python
  teleport_circuit.h(qreg[1])
  teleport_circuit.cx(qreg[1], qreg[2])
  ```

#### Bell State Measurement
- **Mathematical basis**: Projects a two-qubit system onto the Bell basis {|Φ⁺⟩, |Φ⁻⟩, |Ψ⁺⟩, |Ψ⁻⟩}
- **In the code**: Implemented with CNOT followed by H, then measurement
- **Lines of code**:
  ```python
  teleport_circuit.cx(qreg[0], qreg[1])
  teleport_circuit.h(qreg[0])
  teleport_circuit.measure(qreg[0], creg[0])
  teleport_circuit.measure(qreg[1], creg[1])
  ```

### 2. Mathematical Transformations

#### Hadamard Transform
- **Mathematical representation**: H = (1/√2) * [[1, 1], [1, -1]]
- **Effect**: Transforms basis states into superpositions:
  - |0⟩ → (|0⟩ + |1⟩)/√2 = |+⟩
  - |1⟩ → (|0⟩ - |1⟩)/√2 = |-⟩

#### Controlled-NOT (CNOT) Operation
- **Mathematical representation**: CNOT = [[1,0,0,0], [0,1,0,0], [0,0,0,1], [0,0,1,0]]
- **Effect**: Flips the target qubit if the control qubit is |1⟩

#### Pauli-X (NOT) Gate
- **Mathematical representation**: X = [[0,1], [1,0]]
- **Effect**: Bit flip operation: |0⟩ → |1⟩ and |1⟩ → |0⟩

#### Pauli-Z Gate
- **Mathematical representation**: Z = [[1,0], [0,-1]]
- **Effect**: Phase flip operation: |0⟩ → |0⟩ and |1⟩ → -|1⟩

### 3. Algorithms and Protocols

#### Quantum Teleportation Protocol
This is the core algorithm implemented in the code, which follows these steps:

1. **State Preparation**: Create the state to be teleported
2. **Entanglement Generation**: Create an entangled Bell pair between qubits A and B
3. **Bell Measurement**: Perform a joint measurement on the input state and qubit A
4. **Classical Communication**: (Simulated by conditional operations in code)
5. **Conditional Unitary Operations**: Apply X and/or Z gates based on measurement results

#### Conditional Logic
- **Mathematical representation**: If-then logic based on measurement outcomes
- **In the code**: Implemented using the `.c_if()` method to apply gates conditionally
- **Lines of code**:
  ```python
  teleport_circuit.x(qreg[2]).c_if(creg[1], 1)
  teleport_circuit.z(qreg[2]).c_if(creg[0], 1)
  ```

## 📊 Running on a Real Quantum Computer (Optional)

If you have access to IBM Quantum, you can run the teleportation circuit on a real quantum computer:

In [ ]:


 from qiskit import IBMQ
 IBMQ.save_account('YOUR_API_TOKEN')
 provider = IBMQ.load_account()

# Get the least busy backend with at least 3 qubits
# from qiskit.providers.ibmq import least_busy
# small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits >= 3 and 
#                                              not x.configuration().simulator)
# backend = least_busy(small_devices)
# print(f"Using real quantum computer: {backend.name()}")

# # Execute the circuit on a real quantum computer
# job = execute(teleport_circuit, backend, shots=1024)
# result = job.result()
# counts = result.get_counts(teleport_circuit)

# # Display the results
# print("Real Quantum Computer Results:")
# print(counts)
# plot_histogram(counts)

## 🧩 Conclusion

In this notebook, we've implemented and analyzed the quantum teleportation protocol using Qiskit. We've seen how:

1. Quantum teleportation allows us to transfer quantum states between qubits without violating the no-cloning theorem
2. Entanglement serves as the quantum resource that enables teleportation
3. Classical communication is still needed to complete the teleportation process
4. The protocol combines multiple quantum operations and measurements in a specific sequence

Quantum teleportation is not only a fascinating demonstration of quantum mechanics but also a foundational building block for quantum networks and distributed quantum computing.

## 🔍 Further Explorations

Here are some ideas to extend this project:

1. Teleport different initial states (not just |+⟩) and verify the results
2. Implement quantum teleportation for multi-qubit systems
3. Explore the effects of noise on teleportation fidelity
4. Compare results between simulators and real quantum hardware
5. Implement remote state preparation (a related protocol)